In [1]:
import torch as th 
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import pytorch_lightning as pl 
from pytorch_lightning import Trainer, LightningModule

import torchdyn 
from torchdyn.core import NeuralODE

import os
import cv2 
import numpy as np 
import matplotlib.pyplot as plt 

import warnings 
warnings.filterwarnings("ignore")

In [2]:
PATH = r'/Users/suyashsachdeva/Desktop/GyanBhandar/CircuitSolver/'

In [8]:
class CircuitFused(Dataset):
    def __init__(self, datadir, image_size):
        super().__init__()
        self.datadir = datadir
        self.image_size = image_size
        self.images, self.labels = self.datareader()

    def datareader(self):
        X = []
        Y = []
        folders = os.listdir(self.datadir)
        try:
            for folder in folders:
                files = os.listdir(self.datadir+folder)
                for file in files:
                    image = cv2.imread(self.datadir+folder+"/"+file)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                    image = cv2.resize(image, (self.image_size[0], self.image_size[1]), cv2.INTER_AREA)/255.0
                    X.append(image.reshape(1, self.image_size[0], self.image_size[1]))
                    Y.append(folder)
        except:
            pass
        return X, Y

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        label = self.labels[idx]
        image = self.images[idx]
        sample = {"Image": image, "Label": label}
        return sample

In [15]:
CircuitDataset = DataLoader(CircuitFused(PATH, (64, 64)), batch_size=100, shuffle=True)

In [ ]:
class CNNODEBlock(nn.Module):
    def __init__(self, infilter, outfilter, kernel, moment):
        super(CNNODEBlock, self).__init__()
        self.conv1 = nn.Conv2d(infilter, outfilter, kernel)
        self.act = nn.ReLU()
        self.conv2 = nn.Conv2d(outfilter, infilter, kernel)
        pad = int((kernel-1)/2)
        self.pad = nn.ZeroPad2d(pad)
        self.norm1 = nn.BatchNorm2d(outfilter, momentum=moment)
        self.norm2 = nn.BatchNorm2d(infilter, momentum=moment)

    def forward(self, x):
        x = self.act(self.norm1(self.pad(self.conv1(x))))
        return self.act(self.norm2(self.pad(self.conv2(x))))
    
class UPBlock(nn.Module):
    def __init__(self, infilter, outfilter, kernel, moment):
        super(UPBlock).__init__()
        self.conv1 = nn.Conv2d()
        

class CNNODE(nn.Module):
    def __init__(self, num=3, filter=64, dense=256, classes=15):
        super(CNNODE, self).__init__()
        self
t_span = th.linspace(0, 1, 100)